# HSE 2024: Mathematical Methods for Data Analysis

## Homework 2

# Attention!

* For tasks where <ins>text answer</ins> is required **Russian language** is **allowed**.
* If a task asks you to describe something (make coclusions) then **text answer** is **mandatory** and **is** part of the task
* **Do not** upload the dataset (titanic.csv) to the grading system (we already have it)
* We **only** accept **ipynb** notebooks. If you use Google Colab then you'll have to download the notebook before passing the homework
* **Do not** use python loops instead of NumPy vector operations over NumPy vectors - it significantly decreases performance (see why https://blog.paperspace.com/numpy-optimization-vectorization-and-broadcasting/), will be punished with -0.25 for **every** task.
Loops are only allowed in part 1 (Tasks 1 - 4).
* Some tasks contain tests. They only test you solution on a simple example, thus, passing the test does **not** guarantee you the full grade for the task.

If the task asks for an explanation of something, it means that a written answer is required, which is part of the task and is assessed

We only accept ipynb notebooks. If you use Google Colab, you need to download the notebook before submitting your homework

In [130]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn import datasets
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLSResults
from math import sqrt
import random
import sys

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

sns.set(style="darkgrid")

### Data

For this homework we will use a dataset of tracks from the streaming service Spotify

**Описание данных**

- **track_id:** The Spotify ID for the track
- **artists:** The artists' names who performed the track. If there is more than one artist, they are separated by a ;
- **album_name:** The album name in which the track appears
- **track_name:** Name of the track
- **popularity:** The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity.
- **duration_ms:** The track length in milliseconds
- **explicit:** Whether or not the track has explicit lyrics (true = yes it does; false = no it does not OR unknown)
- **danceability:** Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable
- **key:** The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. If no key was detected, the value is -1
- **loudness:** The overall loudness of a track in decibels (dB)
- **mode:** Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0
- **speechiness:** Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks
- **acousticness:** A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic
- **instrumentalness:** Predicts whether a track contains no vocals. "Ooh" and "aah" sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly "vocal". The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content
- **liveness:** Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live
- **valence:** A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry)
- **tempo:** The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration
- **time_signature:** An estimated time signature. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). The time signature ranges from 3 to 7 indicating time signatures of 3/4, to 7/4.
- **track_genre:** The genre in which the track belongs

**Target variable**
- **energy:** Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale

In [131]:
data = pd.read_csv('dataset.csv')

y = data['energy']
X = data.drop(['energy'], axis=1)
columns = X.columns


## Linear Regression

#### 0. [0.25 points] Code the categorical features. Explain the method you have chosen.

In [132]:
data.info()

In [133]:
data.isna().sum()

In [134]:
data.dropna()

In [135]:
num_col = X.select_dtypes([np.number]).columns
print(num_col)
cat_col = X.select_dtypes(include=['object']).columns
cat_col


In [136]:
for i in cat_col:
    a = X[i].unique()
    print(i)
    print(len(a))
    print(a)

Я решил закодировать только переменную track_genre, так как остальные переменные имеют большое кол-во разных значений и поэтому наша модель может из-за этого переобучиться

In [137]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False)
data_one = encoder.fit_transform(X[['track_genre']])
genre_columns = encoder.get_feature_names_out(['track_genre'])
print(genre_columns)
X = X.join(pd.DataFrame(data_one, columns=encoder.get_feature_names_out(['track_genre'])))

X.head(10)

#### 1. [0.25 points] Split the data into train and test with a ratio of 80:20 and random_state=42.

In [138]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### 2. [0.75 points] Train models on train, excluding categorical features, using the StatsModels library and apply it to test; use $RMSE$ and $R^2$ as quality metrics. Try also applying linear regression implementations from sklearn:

* [`LinearRegression`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html);
* [`Ridge`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html) with $\alpha = 0.03$;
* [`Lasso`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) with $\alpha = 0.05$

Don't forget to scale your data using StandardScaler before training your models!

In [139]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train[num_col])
X_test_scaled = scaler.fit_transform(X_test[num_col])

model_1 = LinearRegression()
model_1.fit(X_train_scaled, y_train)
y_pred_1 = model_1.predict(X_test_scaled)
print('Test RMSE = %.3f on Linear Regression' % mean_squared_error(y_test, y_pred_1, squared=False))
print('R^2 = %.3f on Linear Regression' % model_1.score(X_test_scaled, y_test))
print()

model_2 = Ridge(alpha=0.03)
model_2.fit(X_train_scaled, y_train)
y_pred_2 = model_2.predict(X_test_scaled)
print('Test RMSE = %.3f on Ridge with alpha = 0.03' % mean_squared_error(y_test, y_pred_2, squared=False))
print('R^2 = %.3f on Ridge with alpha = 0.03' % model_2.score(X_test_scaled, y_test))
print()


model_3 = Lasso(alpha=0.05)
model_3.fit(X_train_scaled, y_train)
y_pred_3 = model_3.predict(X_test_scaled)
print('Test RMSE = %.3f on Lasso with alpha = 0.05' % mean_squared_error(y_test, y_pred_3, squared=False))
print('R^2 = %.3f on Lasso with alpha = 0.05' % model_3.score(X_test_scaled, y_test))
print()


#### 3. [0.25 points] Repeat the steps from the previous point, adding categorical features. Comment on the changes in the quality metrics values

In [140]:
scaler1 = StandardScaler()

X_train_scaled = scaler1.fit_transform(X_train.select_dtypes([np.number]))
X_test_scaled = scaler1.fit_transform(X_test.select_dtypes([np.number]))

model_1 = LinearRegression()
model_1.fit(X_train_scaled, y_train)
y_pred_1 = model_1.predict(X_test_scaled)
print('Test RMSE = %.3f on Linear Regression' % mean_squared_error(y_test, y_pred_1, squared=False))
print('R^2 = %.3f on Linear Regression' % model_1.score(X_test_scaled, y_test))
print()

model_2 = Ridge(alpha=0.03)
model_2.fit(X_train_scaled, y_train)
y_pred_2 = model_2.predict(X_test_scaled)
print('Test RMSE = %.3f on Ridge with alpha = 0.03' % mean_squared_error(y_test, y_pred_2, squared=False))
print('R^2 = %.3f on Ridge with alpha = 0.03' % model_2.score(X_test_scaled, y_test))
print(model_2)


model_3 = Lasso(alpha=0.05)
model_3.fit(X_train_scaled, y_train)
y_pred_3 = model_3.predict(X_test_scaled)
print('Test RMSE = %.3f on Lasso with alpha = 0.05' % mean_squared_error(y_test, y_pred_3, squared=False))
print('R^2 = %.3f on Lasso with alpha = 0.05' % model_3.score(X_test_scaled, y_test))
print()


Как видим результат у линейной регрессии не особо впечатляющий, а вызвано это тем, что модель переобучилась из-за большого числа признаков, в то время как Ridge показал лучший результат, а Lasso такой же, так как они являются  регуляризирующими функциями

#### 4. [1 point] Examine the parameter values ​​of the models obtained from StatsModels and check which weights and in which models turned out to be zero. Comment on the significance of the coefficients, the overall significance of the models and other factors from the resulting tables

In [141]:
linear_zeros = np.sum(model_1.coef_ == 0)
ridge_zeros = np.sum(model_2.coef_ == 0)
lasso_zeros = np.sum(model_3.coef_ == 0)
print("Zero weights in Linear:", ridge_zeros)
print("Zero weights in Ridge:", ridge_zeros)
print("Zero weights in Lasso:", lasso_zeros)

#### 5. [1 point] Implement one of the feature selection algorithms (Elimination by P-value, Forward elimination, Backward elimination), draw conclusions.

In [142]:
# your code here
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

#### 6. [1 point] Find the best (RMSE) $\alpha$ for Lasso regression using 4-fold cross-validation. You should choose a value from the logarithmic range $[10^{-4}, 10^{3}]$.

In [143]:
alphas = np.logspace(-4, 3, 20)
searcher = GridSearchCV(Lasso(), [{"alpha": alphas}], scoring="neg_root_mean_squared_error", cv=4)
searcher.fit(X_train_scaled, y_train)

best_alpha = searcher.best_params_["alpha"]
print("Best alpha = %.4f" % best_alpha)

plt.plot(alphas, -searcher.cv_results_["mean_test_score"])
plt.xscale("log")
plt.xlabel("alpha")
plt.ylabel("CV score")

## Gradient Descent

#### 7. [3.5 points] Implement Ridge regression for MSE loss trained using gradient descent.

All computations must be vectorized, and Python loops can only be used for gradient descent iterations. The stopping criteria must be (simultaneously):

* checking the absolute norm of the difference in weights on two adjacent iterations (e.g., less than some small number of the order of $10^{-6}$, specified by the `tolerance` parameter);

* reaching the maximum number of iterations (e.g., 10000, specified by the `max_iter` parameter).

You need to do:

* Full gradient descent:

$$
w_{k + 1} = w_{k} - \eta_{k} \nabla_{w} Q(w_{k}).
$$

* Stochastic Gradient Descent:

$$
w_{k + 1} = w_{k} - \eta_{k} \nabla_{w} q_{i_{k}}(w_{k}).
$$

$\nabla_{w} q_{i_{k}}(w_{k}) \, $ is an estimate of the gradient over a set of objects chosen at random.

* Moment of method:

$$
h_0 = 0, \\
h_{k + 1} = \alpha h_{k} + \eta_k \nabla_{w} Q(w_{k}), \\
w_{k + 1} = w_{k} - h_{k + 1}.
$$

* Adagrad method:

$$
G_0 = 0, \\
G_{k + 1} = G_{k} + (\nabla_{w} Q(w_{k+1}))^2, \\
w_{k + 1} = w_{k} - \eta * \frac{\nabla_{w} Q(w_{k+1})}{\sqrt{G_{k+1} + \epsilon}}.
$$

To verify that the optimization process is actually running, we will use the `loss_history` class attribute. After calling the fit method, it should contain the loss function values ​​for all iterations starting from the first (up to the first step along the antigradient).

You need to initialize the weights with a random vector from a normal distribution. Below is a template that should contain code implementing all the model variants.

In [144]:
from sklearn.base import BaseEstimator

class LinReg(BaseEstimator):
    def __init__(self, delta=1.0, gd_type='Momentum',
                 tolerance=1e-4, max_iter=1000, w0=None, eta=1e-2, alpha=1e-2):
        """
        gd_type: str
            'GradientDescent', 'StochasticDescent', 'Momentum', 'Adagrad'
        delta: float
            proportion of object in a batch (for stochastic GD)
        tolerance: float
            for stopping gradient descent
        max_iter: int
            maximum number of steps in gradient descent
        w0: np.array of shape (d)
            init weights
        eta: float
            learning rate
        alpha: float
            momentum coefficient
        """

        self.delta = delta
        self.gd_type = gd_type
        self.tolerance = tolerance
        self.max_iter = max_iter
        self.w0 = w0
        self.alpha = alpha
        self.w = None
        self.eta = eta
        self.loss_history = None # list of loss function values at each training iteration
        self.epsilon = 1e-8

    def fit(self, X, y):
        """
        X: np.array of shape (l, d)
        y: np.array of shape (l)
        ---
        output: self
        """
        self.loss_history = []
        
        l, d = X.shape
        self.w = np.random.randn(d) if self.w0 is None else self.w0
        
        momentum = np.zeros(d)
        G = np.zeros(d)
        n_splits = 10 # на сколько частей делится выборка при стох. град. спуске
        batch_size = l // n_splits
        for i in range(self.max_iter):
            if self.gd_type == 'GradientDescent':
                grad = self.calc_gradient(X, y)
                self.w -= self.eta * grad
                
            elif self.gd_type == 'StochasticDescent':
                for j in range(0, l, batch_size):
                    X_batch = X[j:j + batch_size]
                    y_batch = y[j:j + batch_size]
                    grad = self.calc_gradient(X_batch, y_batch)
                    self.w -= self.eta * grad
                
            elif self.gd_type == 'Momentum':
                grad = self.calc_gradient(X, y)
                momentum = self.alpha * momentum + self.eta * grad
                self.w -= momentum
            
            elif self.gd_type == 'Adagrad':
                grad = self.calc_gradient(X, y)
                G += grad**2
                self.w -= (self.eta / (np.sqrt(G + self.epsilon))) * grad
            
            
            loss = self.calc_loss(X, y)
            self.loss_history.append(loss)
        
            if i > 0 and np.linalg.norm(self.loss_history[-2] - loss) < self.tolerance:
                break
        
        return self

    def predict(self, X):
        if self.w is None:
            raise Exception('Not trained yet')
        return X.dot(self.w)

    def calc_gradient(self, X, y):
        """"
        X: np.array of shape (l, d) (l can be equal to 1 if stochastic)
        y: np.array of shape (l)
        ---
        output: np.array of shape (d)
        """
        l = X.shape[0]
        y_pred = X.dot(self.w)
        grad = -2 / l * X.T.dot(y - y_pred)  # Gradient of MSE
        return grad

    def calc_loss(self, X, y):
        """
        X: np.array of shape (l, d)
        y: np.array of shape (l)
        ---
        output: float
        """
        y_pred = X.dot(self.w)
        loss = np.mean((y - y_pred) ** 2)  # MSE loss
        return loss
        

#### 8. [1 point] Train and validate "manual" models on the same data, compare the quality with models from Sklearn and StatsModels. Investigate the influence of the `max_iter` and `alpha` parameters on the optimization process. Does it meet your expectations?

In [145]:
model_1 = LinReg(gd_type='GradientDescent')
model_1.fit(X_train_scaled, y_train)
y_pred_1 = model_1.predict(X_test_scaled)

y_mean = np.mean(y_test)
ss_res = np.sum((y_test - y_pred_1) ** 2)
ss_tot = np.sum((y_test - y_mean) ** 2)
r2 = 1 - (ss_res / ss_tot)

print('Test RMSE = %.3f on GradientDescent' % mean_squared_error(y_test, y_pred_1, squared=False))
print('R^2 = %.3f on GradientDescent' % r2)
print()


In [146]:
model_2 = LinReg(gd_type='StochasticDescent')
model_2.fit(X_train_scaled, y_train)
y_pred_2 = model_2.predict(X_test_scaled)

y_mean = np.mean(y_test)
ss_res = np.sum((y_test - y_pred_2) ** 2)
ss_tot = np.sum((y_test - y_mean) ** 2)
r2 = 1 - (ss_res / ss_tot)

print('Test RMSE = %.3f on StochasticDescent' % mean_squared_error(y_test, y_pred_2, squared=False))
print('R^2 = %.3f on StochasticDescent' % r2)
print()

In [147]:
model_3= LinReg(gd_type='Momentum')
model_3.fit(X_train_scaled, y_train)
y_pred_3 = model_3.predict(X_test_scaled)

y_mean = np.mean(y_test)
ss_res = np.sum((y_test - y_pred_3) ** 2)
ss_tot = np.sum((y_test - y_mean) ** 2)
r2 = 1 - (ss_res / ss_tot)

print('Test RMSE = %.3f on Momentum' % mean_squared_error(y_test, y_pred_3, squared=False))
print('R^2 = %.3f on Momentum' % r2)
print()

In [148]:
model_4 = LinReg(gd_type='Adagrad')
model_4.fit(X_train_scaled, y_train)
y_pred_4 = model_4.predict(X_test_scaled)

y_mean = np.mean(y_test)
ss_res = np.sum((y_test - y_pred_4) ** 2)
ss_tot = np.sum((y_test - y_mean) ** 2)
r2 = 1 - (ss_res / ss_tot)

print('Test RMSE = %.3f on Adagrad' % mean_squared_error(y_test, y_pred_4, squared=False))
print('R^2 = %.3f on Adagrad' % r2)
print()

Можем наблюдать, что результат намного ниже, чем у scikit-learn

#### 9. [1 point] Plot graphs of the loss function values ​​as a function of the iteration number for all models (full gradient descent, stochastic gc, Momentum, and Adagrad). Draw conclusions about the convergence rate of various modifications of gradient descent.

Don't forget about what a *nice* graph should look like!

In [149]:
model_1 = LinReg(gd_type='GradientDescent', max_iter=200)
model_1.fit(X_train_scaled, y_train)

model_2 = LinReg(gd_type='StochasticDescent', max_iter=200)
model_2.fit(X_train_scaled, y_train)

model_3 = LinReg(gd_type='Momentum', max_iter=200)
model_3.fit(X_train_scaled, y_train)

model_4 = LinReg(gd_type='Adagrad', max_iter=200)
model_4.fit(X_train_scaled, y_train)

plt.plot(model_1.loss_history, label='Full Gradient Descent')
plt.plot(model_2.loss_history, label='Stochastic Gradient Descent')
plt.plot(model_3.loss_history, label='Momentum')
plt.plot(model_4.loss_history, label='Adagrad')

plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.title('Convergence of Different Gradient Descent Methods')
plt.legend()
plt.grid(True)
plt.show()

Из графика видим, что быстрее всех справляется с задачей стохастический градиентный спуск, а медленнее всех adagrad

In [150]:
model_1 = LinReg(gd_type='GradientDescent', max_iter=2000)
model_1.fit(X_train_scaled, y_train)

model_2 = LinReg(gd_type='StochasticDescent', max_iter=2000)
model_2.fit(X_train_scaled, y_train)

model_3 = LinReg(gd_type='Momentum', max_iter=2000)
model_3.fit(X_train_scaled, y_train)

model_4 = LinReg(gd_type='Adagrad', max_iter=2000)
model_4.fit(X_train_scaled, y_train)

plt.plot(model_1.loss_history, label='Full Gradient Descent')
plt.plot(model_2.loss_history, label='Stochastic Gradient Descent')
plt.plot(model_3.loss_history, label='Momentum')
plt.plot(model_4.loss_history, label='Adagrad')

plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.title('Convergence of Different Gradient Descent Methods')
plt.legend()
plt.grid(True)
plt.show()

Как видим при большем количестве итераций, adagrad значительно проигрывает остальным методам, или я его неправильно реализовал)